# Resampling

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Data loading and splitting

In [12]:
Auto = load_data('Auto')
# int(Auto.shape[0] / 2) can replace with 196
Auto_train,Auto_valid = train_test_split(Auto, test_size = int(Auto.shape[0] / 2), random_state=0)


### Training the model with training data split

In [24]:
hp_mm = MS(['horsepower'])
X_train =hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model =sm.OLS(y_train, X_train)
results = model.fit()

### Evaluate the model matrix for this model created using the validation data set

In [25]:
X_valid = hp_mm.transform(Auto_valid)
y_valid =Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)


23.61661706966988